In [ ]:
import time
import numpy as np
import pandas as pd
from tqdm import tqdm
from IPython.display import display
import datetime
import contextlib
import io
import import_ipynb

import enhanced_genetic_algorithm as ega
import particle_swarm_optimization as pso
import non_dominated_sorting_genetic_algorithm_2 as nsga
import utils.fitness_functions as ff

RUNS = 5
DATASETS = {
    "Small": "data/dataset_small.csv",
    "Medium": "data/dataset_medium.csv",
    "Large": "data/dataset_large.csv"
}

results = []

for label, file_path in DATASETS.items():

    dataset_df = pd.read_csv(file_path)

    timestamp = datetime.datetime.now().strftime("run_%Y%m%d_%H%M%S")
    ega.RUN_TIME = timestamp
    ega.BASE_DIR = "output/statistics/ega"
    ega.DATASET = dataset_df

    timestamp = datetime.datetime.now().strftime("run_%Y%m%d_%H%M%S")
    pso.RUN_TIME = timestamp
    pso.BASE_DIR = "output/statistics/pso"
    pso.DATASET = dataset_df

    timestamp = datetime.datetime.now().strftime("run_%Y%m%d_%H%M%S")
    nsga.RUN_TIME = timestamp
    nsga.BASE_DIR = "output/statistics/nsga_ii"
    nsga.DATASET = dataset_df

    ga_times, ga_attempts = [], []
    pso_times, pso_attempts = [], []
    nsga_times, nsga_attempts = [], []

    for _ in tqdm(range(RUNS), desc=label):
        with contextlib.redirect_stdout(io.StringIO()):
            start = time.time()
            df_ga, _ = ega.execute()
            ga_times.append(time.time() - start)
            ga_attempts.append(df_ga["number of computations"].iloc[-1])

            start = time.time()
            df_pso, _ = pso.execute()
            pso_times.append(time.time() - start)
            pso_attempts.append(df_pso["number of computations"].iloc[-1])

            start = time.time()
            _, log = nsga.execute_nsga_ii()
            nsga_times.append(time.time() - start)
            nsga_attempts.append(log[max(log)])

    results.append({
        "Dataset": label,
        "GA Time (s)": round(np.mean(ga_times), 2),
        "GA Computations": round(np.mean(ga_attempts)),
        "GA Computations Std": round(np.std(ga_attempts)),
        "PSO Time (s)": round(np.mean(pso_times), 2),
        "PSO Computations": round(np.mean(pso_attempts), 2),
        "PSO Computations Std": round(np.std(pso_attempts)),
        "NSGA-II Time (s)": round(np.mean(nsga_times), 2),
        "NSGA-II Computations": round(np.mean(nsga_attempts)),
        "NSGA-II Computations Std": round(np.std(nsga_attempts))
    })

# Display results as table
df = pd.DataFrame(results)
display(df)
